In [8]:
pip install langgraph langchain langchain-ollama

In [9]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# The State dictates what data flows through the graph.
# Here, we just track the list of messages in the conversation.
class AgentState(TypedDict):
    messages: List[BaseMessage]
    category: str # We will store the classification here (Math vs General)

In [10]:
# LLM setup: prefer Ollama, fall back to OpenAI if unavailable
import os
from typing import Optional
from langchain_core.messages import AIMessage

llm = None
llm_provider: Optional[str] = None

# Try Ollama first
try:
    from langchain_ollama import ChatOllama
    # Note: the `ollama` CLI must be installed and running for remote calls to succeed
    llm = ChatOllama(model="llama3", temperature=0)
    llm_provider = "ollama"
    print('Using Ollama backend for LLM')
except Exception as e:
    print('Ollama unavailable:', e)
    # Try OpenAI via LangChain as a fallback (requires OPENAI_API_KEY in env)
    try:
        from langchain.chat_models import ChatOpenAI
        if os.environ.get('OPENAI_API_KEY'):
            llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
            llm_provider = "openai"
            print('Using OpenAI fallback for LLM')
        else:
            print('OPENAI_API_KEY not set; OpenAI fallback disabled')
    except Exception as e2:
        print('OpenAI LangChain client unavailable:', e2)

def send_prompt(prompt: str) -> str:
    """Send a prompt to the available LLM and return plain text."""
    if llm is None:
        raise RuntimeError('No LLM available. Install Ollama or set OPENAI_API_KEY for OpenAI fallback.')
    # Ollama client exposes `invoke` which returns a message-like object with `content`
    if llm_provider == 'ollama':
        resp = llm.invoke(prompt)
        return getattr(resp, 'content', str(resp)).strip()
    # For LangChain ChatOpenAI try common call patterns
    try:
        # Some LangChain LLMs implement `predict` which returns a string
        if hasattr(llm, 'predict'):
            return llm.predict(prompt).strip()
        # Fallback to calling the object directly
        out = llm(prompt)
        # If it returns an LLMResult-like object, try to extract text
        if hasattr(out, 'generations'):
            gens = out.generations
            if gens and gens[0] and hasattr(gens[0][0], 'text'):
                return gens[0][0].text.strip()
        return str(out).strip()
    except Exception as e:
        raise

Using Ollama backend for LLM


In [11]:
# Node 1: The Categorizer
def categorize_input(state: AgentState):
    """
    Analyzes the user's last message and decides if it's 'Math' or 'General'.
    """
    last_message = state["messages"][-1].content
    prompt = f"You are a router. Classify the following input as either 'Math' or 'General'. Return ONLY the word 'Math' or 'General'. Do not add punctuation.\n\nInput: {last_message}"
    try:
        category_text = send_prompt(prompt)
        category = category_text.strip()
    except Exception as e:
        print('Categorization failed:', e)
        # Default to General if classification fails
        category = 'General'
    return {"category": category}

# Node 2: The Math Expert
def handle_math(state: AgentState):
    print("--- 🧮 Entering Math Node ---")
    last_message = state["messages"][-1].content
    try:
        text = send_prompt(f"You are a mathematician. Solve this simply: {last_message}")
    except Exception as e:
        print('Math node LLM error:', e)
        text = f"Error: {e}"
    return {"messages": [AIMessage(content=text)]}

# Node 3: The General Chat
def handle_general(state: AgentState):
    print("--- 💬 Entering General Chat Node ---")
    last_message = state["messages"][-1].content
    try:
        text = send_prompt(f"You are a helpful assistant. Reply to: {last_message}")
    except Exception as e:
        print('General node LLM error:', e)
        text = f"Error: {e}"
    return {"messages": [AIMessage(content=text)]}

In [12]:
def routing_logic(state: AgentState):
    # If the category is Math, go to 'math_node'
    if state["category"] == "Math":
        return "math_node"
    # Otherwise, go to 'general_node'
    return "general_node"

In [13]:
from langgraph.graph import StateGraph, END

# 1. Initialize the Graph with our State structure
workflow = StateGraph(AgentState)

# 2. Add the Nodes
workflow.add_node("categorizer", categorize_input)
workflow.add_node("math_node", handle_math)
workflow.add_node("general_node", handle_general)

# 3. Set the Entry Point
# When the graph starts, the first person to touch the ball is the 'categorizer'
workflow.set_entry_point("categorizer")

# 4. Add Conditional Edges
# After 'categorizer' runs, look at 'routing_logic' to decide where to go next.
workflow.add_conditional_edges(
    "categorizer",        # From this node...
    routing_logic,        # ...run this logic...
    {                     # ...and map the output to a node.
        "math_node": "math_node",
        "general_node": "general_node"
    }
)

# 5. Add Normal Edges
# After math or general chat, we are done. Go to END.
workflow.add_edge("math_node", END)
workflow.add_edge("general_node", END)

# 6. Compile
app = workflow.compile()

In [14]:
# Test 1: A Math Question
print("\n--- TEST 1: Math ---")
inputs_1 = {"messages": [HumanMessage(content="What is 55 multiplied by 10?")]}

# Stream the output to see the steps
for event in app.stream(inputs_1):
    for key, value in event.items():
        print(f"Finished running: {key}")

# Test 2: A Casual Greeting
print("\n--- TEST 2: General ---")
inputs_2 = {"messages": [HumanMessage(content="Tell me a fun fact about history.")]}

for event in app.stream(inputs_2):
    for key, value in event.items():
        print(f"Finished running: {key}")


--- TEST 1: Math ---
Categorization failed: [Errno 111] Connection refused
Finished running: categorizer
--- 💬 Entering General Chat Node ---
General node LLM error: [Errno 111] Connection refused
Finished running: general_node

--- TEST 2: General ---
Categorization failed: [Errno 111] Connection refused
Finished running: categorizer
--- 💬 Entering General Chat Node ---
General node LLM error: [Errno 111] Connection refused
Finished running: general_node
